In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px

from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, LassoCV,RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import mlflow.sklearn
import mlflow

import joblib
import os

# chargement des données

In [ ]:
df = pd.read_csv('../Data/get_around_prix_propre.csv')
df.head(5)

In [ ]:
df.drop(columns=['catégorie','type_km'],axis=1, inplace=True)
df.info()

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Métriques Getaround")

Afin de pouvoir mettre les colonnes booléennes directement en catégorielles, je converti les True/False en Oui/Non

In [5]:
for col in df.select_dtypes(include='bool').columns:
    df[col] = df[col].replace({True: 'Oui', False: 'Non'})


Préparations variables et train_test_split_

In [6]:
target = 'rental_price_per_day'
Y = df[target]
X = df.drop(target, axis=1)

### Initialisation variables pour que ce soit identique soit les essais

In [7]:
random = 42
size = 0.2
nb_CV = 5
nb_alphas = 100
alphas = np.logspace(-3, 2, nb_alphas)
iter = 50000

all_results = pd.DataFrame()
numerical_features = X.select_dtypes(include='number').columns
categorical_features = X.select_dtypes(exclude='number').columns

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = size, random_state = random)

### preprocessing

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first',handle_unknown='ignore')) 
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

column_names = []
for name, pipeline, features_list in preprocessor.transformers_: 
    if name == 'num': 
        features = features_list 
    else: 
        features = pipeline.named_steps['encoder'].get_feature_names_out() 
    column_names.extend(features) 


## Méthode d'évaluation

In [10]:
def evaluate_model(model_name, model, X_train, X_test, Y_train, Y_test, all_results, column_names):

    if hasattr(model, "best_estimator_"):
        best_model = model.best_estimator_
        best_params = model.best_params_
        model_name = f"{model_name} (best)"
    else:
        best_model = model
        best_params = None
        
    best_model.fit(X_train, Y_train)
    Y_train_pred = best_model.predict(X_train)
    Y_test_pred = best_model.predict(X_test)
    
    R2_train = r2_score(Y_train, Y_train_pred)
    R2_test = r2_score(Y_test, Y_test_pred)
    MAE_test = mean_absolute_error(Y_test, Y_test_pred)
    MSE_test = mean_squared_error(Y_test, Y_test_pred)
    RMSE_test = np.sqrt(MSE_test)

    with mlflow.start_run(run_name=model_name):
        # Enregistre le pipeline complet
        mlflow.sklearn.log_model(best_model, model_name)
        
        # Log des métriques calculées
        mlflow.log_metric("R2_train", R2_train)
        mlflow.log_metric("R2_test", R2_test)
        mlflow.log_metric("RMSE", RMSE_test)

        mlflow.log_metric("MAE", MAE_test)
           
    new_results = {
        'model': model_name,
        'R2_train': R2_train,
        'R2_test': R2_test,
        'MAE': MAE_test,
        'RMSE': RMSE_test
    }
    
    if hasattr(best_model, "coef_"): 
        coefs = pd.DataFrame({
            'Variable': column_names,
            'Coefficient': best_model.coef_.ravel()
        })
    elif hasattr(best_model, "feature_importances_"):  
        coefs = pd.DataFrame({
            'Variable': column_names,
            'Coefficient': best_model.feature_importances_
        })
    else:
        coefs = None

    
    save_dir = os.path.join("..", "model_paths")  # descend d’un niveau
    model_filename = os.path.join(save_dir, f"{model_name}.pkl")

    joblib.dump(best_model, model_filename)

    all_results = pd.concat([all_results, pd.DataFrame([new_results])], ignore_index=True)
    return all_results, coefs


## 1. Régression Linéaire

In [ ]:
regressor = LinearRegression()
all_results, Coef_LR = evaluate_model("LinearRegression", regressor, X_train, X_test, Y_train, Y_test, all_results, column_names)


## 2. RidgeCV

In [ ]:
ridge = RidgeCV(alphas=alphas, cv=nb_CV)
all_results, coefs_Ridge = evaluate_model("RidgeCV", ridge, X_train, X_test, Y_train, Y_test, all_results, column_names)


## 3. LassoCV

In [ ]:
lasso = LassoCV(alphas=alphas, cv=nb_CV, max_iter=iter)
all_results, coefs_Lasso = evaluate_model("LassoCV", lasso, X_train, X_test, Y_train, Y_test, all_results, column_names)


## 4. RandomForestRegressor avec GridSearchCV

In [ ]:
param_grid = {
    'n_estimators': [200, 400, 600],
    'max_depth': [None, 10, 15, 25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],  
    'bootstrap': [True]                
}
grid = GridSearchCV(
    RandomForestRegressor(random_state=random, n_jobs=-1),
    param_grid=param_grid,
    cv=nb_CV,
    scoring='r2',
    verbose=2
)
grid.fit(X_train, Y_train)

best_model = grid.best_estimator_

all_results, coefs_GCV = evaluate_model("RandomForrestRegressor", best_model, X_train, X_test, Y_train, Y_test, all_results, column_names)

## 5. XGBoost avec GridSearchCV

In [ ]:
param_grid = {
    'n_estimators': [200, 400],           # Nombre d’arbres
    'max_depth': [4, 6, 8],               # Profondeur des arbres
    'learning_rate': [0.03, 0.05, 0.1],   # Permet d'apprendre plus lentement
    'subsample': [0.7, 0.85, 1.0],        # Ratio d’échantillons utilisés à chaque arbre
    'colsample_bytree': [0.7, 0.85, 1.0], # Ratio de colonnes utilisées à chaque arbre
    'reg_alpha': [0, 0.1, 0.5],           # Régularisation L1 (variables inutiles ignorées)
    'reg_lambda': [1, 3, 5]               # Régularisation L2 (stabilise le modèle)
}
grid = GridSearchCV(
    XGBRegressor(objective='reg:squarederror',random_state=random, n_jobs=-1),
    param_grid=param_grid,
    cv=nb_CV,
    scoring='r2',
    verbose=2
)
grid.fit(X_train, Y_train)

best_model = grid.best_estimator_

pipeline_xgb_final = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model)
])

joblib.dump(pipeline_xgb_final, "../model_paths/XGBoost_API.pkl")

all_results, coefs_XGB  = evaluate_model("XGBoost", best_model, X_train, X_test, Y_train, Y_test, all_results, column_names)


In [ ]:
coefs_all = (
    Coef_LR[['Variable', 'Coefficient']]
        .rename(columns={'Coefficient': 'LinearRegression'})
    .merge(
        coefs_Ridge[['Variable', 'Coefficient']].rename(columns={'Coefficient': 'RidgeCV'}),
        on='Variable', how='outer'
    )
    .merge(
        coefs_Lasso[['Variable', 'Coefficient']].rename(columns={'Coefficient': 'LassoCV'}),
        on='Variable', how='outer'
    )
    .merge(
        coefs_GCV[['Variable', 'Coefficient']].rename(columns={'Coefficient': 'RandomForestRegressor'}),
        on='Variable', how='outer'
    )
    .merge(
        coefs_XGB[['Variable', 'Coefficient']].rename(columns={'Coefficient': 'XGBoost'}),
        on='Variable', how='outer'
    )
    )


In [ ]:
coefs_all_norm = coefs_all.copy()

# Normalisation des colonnes numériques (hors Variable)
for col in ['LinearRegression', 'RidgeCV', 'LassoCV', 'RandomForestRegressor', 'XGBoost']:
    if col in coefs_all_norm.columns:
        abs_col = coefs_all_norm[col].abs()
        coefs_all_norm[col] = abs_col / abs_col.sum()

# Importance moyenne normalisée
coefs_all_norm['Importance_moy_norm'] = coefs_all_norm[
    ['LinearRegression', 'RidgeCV', 'LassoCV', 'RandomForestRegressor','XGBoost']
].mean(axis=1)

# Tri décroissant
coefs_all_norm = coefs_all_norm.sort_values(by='Importance_moy_norm', ascending=False)
coefs_all_norm.head(10)


In [ ]:
all_results

## Choix du modèle

 - Linear / Ridge / Lasso <br>
Ces trois modèles linéaires donnent des performances similaires, avec :<br>
R² test environ 73.5% de la variance expliquée. Test et Train équivalent.<br>
MAE ≈ 12.5 € : L'écart moyen prédit à une moyenne d'erreur de 12.5€<br>
RMSE ≈ 17.8 € : Les valeurs importantes ou faibles sont moins bien prédites.<br>
Ces modèles sont limités par les interactions non linéaires.

 - RandomForestRegressor<br>
R² test à 78.2% de la variance expliquée. Léger overfitting vu que le train est à 97%. Le nombre d'arbres peut être la source de cet écart.<br>
Amélioration de la MAE à 10.67€ et réduction de la RMSE à 16.18€.<br>
Le modèle gère mieux les relations entre les variables.

 - XGBoost<br>
Le R² test s'améliore ce qui démontre une meilleure généralisation avec 82% de variance expliquée.<br>
L'overfitting est toujours présent mais réduit avec l'amélioration du R² test malgré une légère réduction du R² train<br>
La MAE passe en-dessous des 10€ la RMSE en-dessous des 15€

Vu l'analyse, le choix se portera sur le modèle XGBoost.<br>


Meilleures features pour le modèle choisi

In [ ]:
coefs_XGB = coefs_XGB.sort_values(by='Coefficient', ascending=False)
top10 = coefs_XGB.head(10)


fig = px.bar(
    top10,
    x="Coefficient",
    y="Variable",
    orientation="h",
    title="Top 10 des features",
    text="Coefficient"
)

fig.update_layout(
    xaxis_title="Importance",
    yaxis_title="Variables",
    yaxis=dict(tickmode="linear")
)

fig.show()

On retrouve dans les variables importantes :
 - La puissance
 - La boite auto
 - le GPS
 - Le système GetAround
 - Le type d'essence

On voit également que la marque compte également